In [1]:
import cv2
import numpy as np

In [4]:
def maybe_resize(img, dims):
    img = np.array(img, dtype=np.float32)
    if len(img.shape) < 3 or img.shape[2] != 3:
        # some images might be grayscale
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if dims != None:
        im_height, im_width, _ = dims
        img = cv2.resize(img, (im_width, im_height), interpolation=cv2.INTER_LINEAR)
    return img

def preprocess(img, dims=None, need_transpose=False):
    img = maybe_resize(img, dims)
    img = np.asarray(img, dtype=np.uint8)
    # transpose if needed
    if need_transpose:
        img = img.transpose([2, 0, 1])
    return img

In [5]:
def convert_image(dst_dir, image, name):
    processed = preprocess(image, None, False)
    # with open(str(dst_dir / name) + ".bytes", "wb") as f:
    #     f.write(processed.tobytes())
    cv2.imwrite(str(dst_dir / name) + ".jpg", processed)
    return name

import tqdm
import pathlib
import multiprocessing as mp
# images_path = pathlib.Path("../coco_512/val2017/")
# images = list(images_path.glob("*.png"))
# dst_dir = pathlib.Path("../coco_512/val2017_bytes/")
# dst_dir.mkdir(exist_ok=True)
# # cache a preprocessed version of the image
# with mp.Pool(4) as p:
#     res = p.imap_unordered(convert_image, images)
#     for i in tqdm.tqdm(res, total=len(images)):
#         pass
    


In [2]:
import os
from coco import COCO
import json
image_dir = pathlib.Path("./val2017")
indir = pathlib.Path(".")
annotate_file = pathlib.Path("./annotations/instances_val2017.json")
# sizes = [(512, "d0"), (640, "d1"), (768, "d2"), (896, "d3"), (1024, "d4")]
sizes = [(300, "onnx")]



# Build directories.
fmt = "png"
def dowork(idx, size):
    img = coco.imgs[idx]
    # Load, upscale, and save image.
    image = cv2.imread(os.path.join(image_in_path, img['file_name']))
    if len(image.shape) < 3 or image.shape[2] != 3:
        # some images might be grayscale
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    image = cv2.resize(image, size, interpolation=cv2.INTER_LINEAR)
    # cv2.imwrite(os.path.join(image_out_path, img['file_name'][0:-3] + fmt), image)
    convert_image(bytes_out_path, image,  img['file_name'])
    # Update image file extension
    img['file_name'] = img['file_name'][0:-3] + fmt
    # Image dimensions
    img['width'] = size[0]
    img['height'] = size[1]
    images.append(img)
    del image
    return img['file_name']

for size, model_name in sizes:
    size = (size, size)
    outdir = pathlib.Path(f"./coco_{size[0]}")
    image_in_path = indir/ image_dir
    image_out_path = outdir/ image_dir
    bytes_out_path = outdir/ (str(image_dir)+"bytes")
    path, fil = annotate_file.parent, annotate_file.name
    annotate_out_path = outdir / path
    annotate_out_file = annotate_out_path / fil
    annotate_in_file = indir / annotate_file 

    image_out_path.mkdir(exist_ok=True, parents=True)
    annotate_out_path.mkdir(exist_ok=True, parents=True)
    bytes_out_path.mkdir(exist_ok=True, parents=True)

    # Read annotations.
    coco = COCO(str(annotate_in_file))

    # Upscale annotations.
    print(f'Upscaling annotations for {size}')
    annotations = []
    for idx in coco.anns:
        ann = coco.anns[idx]
        # Scaling factors
        img = coco.imgs[ann['image_id']]
        sx = size[0] / img['width']
        sy = size[1] / img['height']
        # Bounding boxes
        bb = ann['bbox']
        bb[0] = bb[0] * sx
        bb[1] = bb[1] * sy
        bb[2] = bb[2] * sx
        bb[3] = bb[3] * sy
        # Area
        ann['area'] = ann['area'] * sx * sy
        annotations.append(ann)

    # Upscale images.
    print(f'Upscaling images for {size}')
    count = 0
    images = []
    with mp.Pool(4) as p:
        res = p.starmap(dowork, zip(coco.imgs.keys(), [size]*len(coco.imgs)))
        for _ in tqdm.tqdm(res, total=len(coco.imgs)):
            pass
            

    # Save annotations.
    print('Saving annotations...')
    with open(annotate_in_file) as f:
        dataset = json.load(f)
    dataset['images'] = images
    dataset['annotations'] = annotations
    with open(annotate_out_file, 'w') as outfile:
        json.dump(dataset, outfile)
    print('Done.')

NameError: name 'pathlib' is not defined

In [1]:
import PIL
import matplotlib.pyplot as plt
import numpy as np
import cv2


In [2]:
with open("./coco_512/val2017bytes/000000000139.jpg.bytes", 'rb') as f:
    data = f.read()
    img = np.frombuffer(data, dtype=np.uint8)
    img = img.reshape((512, 512, 3))
    

In [3]:
image = PIL.Image.fromarray(img)

In [5]:
with open("./coco_300/val2017bytes/000000000139.jpg.jpg" , "rb") as f:
    data = f.read()
    img = np.frombuffer(data, dtype=np.uint8)
    # img = img.reshape((300, 300, 3))
    image = cv2.imdecode(img, cv2.IMREAD_COLOR)
    print(image.shape)
    PIL.Image.fromarray(image).show()
    # image.save("./coco_300/val2017bytes/000000000139.jpg.jpg")
    # image.close()

(300, 300, 3)
